In [ ]:
#The AILEA Code Challenge
#Let's make some stock market analysis here :)

In [ ]:
#Packages perhaps necessary to install...
#!pip install alpha_vantage, sqlite3, django

In [1]:
import ftplib
import pandas as pd

def get_symbols():
    #STEP 1: this function retrieves teh symbols of interest from the NASDAQ
    #connection to the server
    #username and passwd required!
    ftp = ftplib.FTP('ftp.nasdaqtrader.com', 'Anonymous', 'guest')
    
    #retrieving the data file into our local directory
    ftp.cwd('/symboldirectory/')
    filename = 'nasdaqlisted.txt'
    ftp.retrbinary('RETR ' + filename, open(filename, 'wb').write)
    ftplib.FTP.quit(ftp) #polite exit
    
    #extract just 50 first ones into data frame
    df = pd.read_csv(filename, sep='|', usecols=['Symbol'])
    #df.sort_values('Symbol') #actually useless as the list seems to be in order anyways
    return df.head(50)

companies = get_symbols()
print(companies)

   Symbol
0    AACG
1    AACI
2   AACIU
3   AACIW
4    AADI
5    AADR
6     AAL
7    AAME
8    AAOI
9    AAON
10   AAPB
11   AAPD
12   AAPL
13   AAPU
14   AATC
15   AAWW
16   AAXJ
17   ABCB
18   ABCL
19   ABCM
20   ABEO
21   ABGI
22   ABIO
23   ABMD
24   ABNB
25   ABOS
26   ABSI
27   ABST
28   ABTX
29   ABUS
30   ABVC
31   ACAB
32  ACABU
33  ACABW
34   ACAC
35  ACACU
36  ACACW
37   ACAD
38   ACAH
39  ACAHU
40  ACAHW
41   ACAX
42  ACAXR
43  ACAXU
44  ACAXW
45    ACB
46   ACBA
47  ACBAU
48  ACBAW
49   ACCD


In [2]:
from alpha_vantage.timeseries import TimeSeries
import time

def get_volumes(companies):
    #STEP 2: here we fetch 5 symbols having highest daily volume
    #by using Alpha Vantage API
    
    API_key = 'TYFN3678G5XYSVMO'
    
    av_data = pd.DataFrame({'symbol': [], 'volume': []})
    ts = TimeSeries(key=API_key, output_format='pandas')
    
    #Alpha Vantage API function get_daily() gives tuple (data, general_info).
    #So here av_data[0] refers to the data only.
    #Hence iloc[0] is its newest date and ['5. volume'] is corresponding trading volume.
    print('Obtaining that trade volume info for you. This may take quite some time... \n Thank you for understanding :) \n')
    for name in companies['Symbol']:
        new_row = pd.DataFrame({'symbol': [name], 'volume': [ts.get_daily(name)[0].iloc[0]['5. volume']]})
        av_data = av_data.append(new_row, ignore_index = True)
        time.sleep(15) #AV allows max 5 queries per minute
    
    #finally we sort and select 5 most traded ones
    print('(finally) Done!')
    return av_data.sort_values('volume', ascending=False, ignore_index=True).head(5)

volumes = get_volumes(companies)
print(volumes)

Obtaining that trade volume info for you. This may take quite some time... 
 Thank you for understanding :) 

(finally) Done!
  symbol      volume
0   AAPL  76957768.0
1    AAL  26922500.0
2    ACB   7381908.0
3   ABNB   4121736.0
4   ACAD   1251847.0


In [3]:
import sqlite3 as sql

def write_to_database(volumes):
    #STEP 3: writing the data into a local database
    #Here we use sqlite as it was already familiar.
    #The table is recreated every time.
    db_connection = sql.connect('nasdaq_data.db')
    c = db_connection.cursor()
    c.execute('''DROP TABLE IF EXISTS most_traded''')
    volumes.to_sql('most_traded', db_connection)
    db_connection.close()
    
write_to_database(volumes)

In [4]:
#this is a temporary cell 
#only to demonstrate that database works
db_connection = sql.connect('nasdaq_data.db')
c = db_connection.cursor()
sql_query = '''SELECT * from most_traded'''
tmp_frame = pd.read_sql(sql_query, db_connection)
db_connection.close()
tmp_frame

,index,symbol,volume
0,0,AAPL,76957768.0
1,1,AAL,26922500.0
2,2,ACB,7381908.0
3,3,ABNB,4121736.0
4,4,ACAD,1251847.0
